In [1]:
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [2]:
df = pd.read_csv("./vax_train.csv")
df["label"][(df["label"] == "Neutral")] = 1
df["label"][(df["label"] == "ProVax")] = 0
df["label"][(df["label"] == "AntiVax")] = 2

C:\Users\ML_RTC\AppData\Local\Temp\ipykernel_2564\1081125046.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][(df["label"] == "Neutral")] = 1
C:\Users\ML_RTC\AppData\Local\Temp\ipykernel_2564\1081125046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][(df["label"] == "ProVax")] = 0
C:\Users\ML_RTC\AppData\Local\Temp\ipykernel_2564\1081125046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"][(

In [3]:
df

,id,tweet,label
0,1329771561671417856,Today Pfizer said it's asking U.S. regulators ...,1
1,1266030447919804420,Good news on development of new #covid19 #vacc...,0
2,1238769307456172034,Once they get a corona vaccine I don’t want to...,0
3,1328814394822705152,Free flu vaccines and COVID-19 tests! https://...,1
4,1254610119876280320,"Bollywood Gupshup,Russian Researchers Will Tes...",0
...,...,...,...
4387,1326106718519128064,"👇👇Not a good practice, not good for public tru...",1
4388,1326136835916095493,Something about this vaccine scares me! Anyone...,2
4389,1326186592847982593,@TinkerChip @matthaig1 They have been working ...,0
4390,1325792316716625920,Fingers crossed this is positive data.. \n\nPf...,0


In [4]:

df.sample(frac=1)
train = df[:4000]
test = df[4000:]

In [5]:
train.to_csv("./Train.csv", index = False)
test.to_csv("./Test.csv", index = False)

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [7]:
from datasets import load_dataset
dataset_train = load_dataset('csv', data_files='./Train.csv')
dataset_train = dataset_train.rename_column("tweet", "text")
dataset_train = dataset_train.remove_columns(["id"])
dataset_train = dataset_train["train"]
tokenized_datasets_train = dataset_train.map(tokenize_function, batched=True)

dataset_test = load_dataset('csv', data_files='./Test.csv')
dataset_test = dataset_test.rename_column("tweet", "text")
dataset_test = dataset_test.remove_columns(["id"])
dataset_test = dataset_test["train"]
tokenized_datasets_test = dataset_test.map(tokenize_function, batched=True)

Using custom data configuration default-b038930c5db4f283


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\ML_RTC\.cache\huggingface\datasets\csv\default-b038930c5db4f283\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\ML_RTC\.cache\huggingface\datasets\csv\default-645b41cb8d16ea02\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets_train= tokenized_datasets_train.remove_columns(["text"])
tokenized_datasets_test = tokenized_datasets_test.remove_columns(["text"])

In [9]:
tokenized_datasets_train = tokenized_datasets_train.rename_column("label", "labels")
tokenized_datasets_test = tokenized_datasets_test.rename_column("label", "labels")

In [10]:
tokenized_datasets_train.set_format("torch")
tokenized_datasets_test.set_format("torch")

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets_train, shuffle=True, batch_size=6, num_workers=4, pin_memory = True)
eval_dataloader = DataLoader(tokenized_datasets_test, batch_size=6, num_workers=4, pin_memory = True)

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [13]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [14]:
import torch
model.to("cuda")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [15]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/2001 [00:00<?, ?it/s]

In [17]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
metric = load_metric("accuracy")
model.eval()
for batch in train_dataloader:
    batch = {k: v.to("cuda") for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()